In [1]:
%pip install openwakeword

  Using cached openwakeword-0.6.0-py3-none-any.whl (60 kB)
     ---------------------------------------- 5.6/5.6 MB 3.4 MB/s eta 0:00:00
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl (46 kB)
  Using cached humanfriendly-10.0-py2.py3-none-any.whl (86 kB)
  Using cached pyreadline3-3.4.1-py3-none-any.whl (95 kB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install pyaudio

     ------------------------------------ 164.1/164.1 kB 979.8 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [17]:
import pyaudio
import numpy as np
from openwakeword.model import Model

# Initialize your trained model
model = Model(wakeword_models=["./Hey_Robin.tflite"],    )

# Audio stream configuration
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000  # Ensure this matches your model's requirements
CHUNK = 1024

# Initialize PyAudio
audio = pyaudio.PyAudio()

# Open audio stream
stream = audio.open(format=FORMAT, channels=CHANNELS,
                    rate=RATE, input=True, frames_per_buffer=CHUNK)

print("Listening for wake word...")

try:
    while True:
        # Read a chunk of data from the audio stream
        data = stream.read(CHUNK)
        
        # play that data
        
        # Convert audio data to numpy array
        audio_data = np.frombuffer(data, dtype=np.int16)
        print(audio_data, audio_data.shape)

        # Preprocess the audio data if needed (e.g., normalization, feature extraction)
        # preprocessed_audio_data = preprocess(audio_data)

        # Use your model to detect the wake word
        if model.predict(audio_data):  # Replace with the actual method to predict
            print("Wake word detected!")
            # Trigger your action here
            break  # Exit the loop or handle the wake word detection as needed

except KeyboardInterrupt:
    print("Interrupted by user")

finally:
    # Close the audio stream
    stream.stop_stream()
    stream.close()
    audio.terminate()


Listening for wake word...
[ 0  0 -1 ...  0  0  0] (1024,)
Wake word detected!


In [3]:
import openwakeword
from openwakeword.model import Model

# One-time download of all pre-trained models (or only select models)
openwakeword.utils.download_models()

# Instantiate the model(s)
model = Model(
    wakeword_models=["./Hey_Robin.tflite"],  # can also leave this argument empty to load all of the included pre-trained models
)

# Get audio data containing 16-bit 16khz PCM audio data from a file, microphone, network stream, etc.
# For the best efficiency and latency, audio frames should be multiples of 80 ms, with longer frames
# increasing overall efficiency at the cost of detection latency
frame = get_audio_frame()

# Get predictions for the frame
prediction = model.predict(frame)

embedding_model.tflite: 100%|██████████| 1.33M/1.33M [00:00<00:00, 2.93MiB/s]
embedding_model.onnx: 100%|██████████| 1.33M/1.33M [00:00<00:00, 2.87MiB/s]
melspectrogram.tflite: 100%|██████████| 1.09M/1.09M [00:00<00:00, 1.97MiB/s]
melspectrogram.onnx: 100%|██████████| 1.09M/1.09M [00:00<00:00, 2.01MiB/s]
silero_vad.onnx: 100%|██████████| 1.81M/1.81M [00:00<00:00, 2.79MiB/s]
alexa_v0.1.tflite: 100%|██████████| 855k/855k [00:00<00:00, 2.23MiB/s]
alexa_v0.1.onnx: 100%|██████████| 854k/854k [00:00<00:00, 2.51MiB/s]
hey_mycroft_v0.1.tflite: 100%|██████████| 860k/860k [00:00<00:00, 2.39MiB/s]
hey_mycroft_v0.1.onnx: 100%|██████████| 858k/858k [00:00<00:00, 935kiB/s] 
hey_jarvis_v0.1.tflite: 100%|██████████| 1.28M/1.28M [00:00<00:00, 2.68MiB/s]
hey_jarvis_v0.1.onnx: 100%|██████████| 1.27M/1.27M [00:00<00:00, 2.76MiB/s]
hey_rhasspy_v0.1.tflite: 100%|██████████| 416k/416k [00:00<00:00, 1.84MiB/s]
hey_rhasspy_v0.1.onnx: 100%|██████████| 204k/204k [00:00<00:00, 1.06MiB/s]
timer_v0.1.tflite: 100%|█

NameError: name 'get_audio_frame' is not defined

In [1]:
%pip install argparse

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Copyright 2022 David Scripka. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Imports
import pyaudio
import numpy as np
from openwakeword.model import Model
import argparse


# Parse input arguments
parser=argparse.ArgumentParser()
parser.add_argument(
    "--chunk_size",
    help="How much audio (in number of samples) to predict on at once",
    type=int,
    default=1280,
    required=False
)
parser.add_argument(
    "--model_path",
    help="The path of a specific model to load",
    type=str,
    default="",
    required=True
)
parser.add_argument(
    "--inference_framework",
    help="The inference framework to use (either 'onnx' or 'tflite'",
    type=str,
    default='tflite',
    required=False
)

args=parser.parse_args()
print(args)

# Get microphone stream
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 16000
CHUNK = args.chunk_size
audio = pyaudio.PyAudio()
mic_stream = audio.open(format=FORMAT, channels=CHANNELS, rate=RATE, input=True, frames_per_buffer=CHUNK)

# Load pre-trained openwakeword models
if args.model_path != "":
    owwModel = Model(wakeword_models=[args.model_path], inference_framework=args.inference_framework)
else:
    owwModel = Model(inference_framework=args.inference_framework)

n_models = len(owwModel.models.keys())

# Run capture loop continuosly, checking for wakewords
if __name__ == "__main__":
    # Generate output string header
    print("\n\n")
    print("#"*100)
    print("Listening for wakewords...")
    print("#"*100)
    print("\n"*(n_models*3))

    while True:
        # Get audio
        audio = np.frombuffer(mic_stream.read(CHUNK), dtype=np.int16)

        # Feed to openWakeWord model
        prediction = owwModel.predict(audio)

        # Column titles
        n_spaces = 16
        output_string_header = """
            Model Name         | Score | Wakeword Status
            --------------------------------------
            """

        for mdl in owwModel.prediction_buffer.keys():
            # Add scores in formatted table
            scores = list(owwModel.prediction_buffer[mdl])
            curr_score = format(scores[-1], '.20f').replace("-", "")

            output_string_header += f"""{mdl}{" "*(n_spaces - len(mdl))}   | {curr_score[0:5]} | {"--"+" "*20 if scores[-1] <= 0.5 else "Wakeword Detected!"}
            """

        # Print results table
        print("\033[F"*(4*n_models+1))
        print(output_string_header, "                             ", end='\r')

usage: ipykernel_launcher.py [-h] [--chunk_size CHUNK_SIZE] --model_path
                             MODEL_PATH
                             [--inference_framework INFERENCE_FRAMEWORK]
ipykernel_launcher.py: error: the following arguments are required: --model_path


SystemExit: 2

C:\Users\77\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3386: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
%tb

SystemExit: 2